In [241]:
import sys
import pandas as pd
from sqlalchemy import text, select, insert, table, column, MetaData
from sqlalchemy.orm import Session
import datetime

sys.path.insert(1, '../')

import notebooks.notebook_local_envs
from app.database.connect_to_db import engine, conn
import app.log_stuff
import logging
import os
user_name = os.getlogin()

In [205]:
user_date = datetime.datetime.now()

def get_max_points():
    '''gets max possible points based on the day it is'''

    query = text("select top 1 * from thresholds where date <= :x order by date desc")
    with engine.connect() as conn:
        logging.debug("Executing max points query")
        max_points = conn.execute(query, {'x': user_date})
        max_points = pd.DataFrame(max_points)

    max_points = max_points.iloc[0,2]
    return max_points

get_max_points()

500

In [263]:
def get_max_grade(input_points, max_possible_points):
    '''take user input points and max points to lookup id for grade and return the grade'''
    grade_percent = input_points / max_possible_points

    query = text("select top 1 * from grade_scale where grade_percent <= :x")

    with engine.connect() as conn:
        logging.debug("Executing grade query")
        grade_scale = conn.execute(query, {'x': grade_percent})
        grade_scale = pd.DataFrame(grade_scale)

    # insert into db
    grade_audit = table("grade_audit",
                        column("username"),
                        column("date"),
                        column("grade_scale_id"))

    # assign index locations to variables, if empty query then id = 12 as fail
    try:
        letter_grade = grade_scale.iloc[0,1]
        gpa = grade_scale.iloc[0,3]
        grade_scale_id = grade_scale.iloc[0,0]
    except IndexError:
        grade_scale_id = 12
        return "You've failed :("
    finally:
        insert_query = (
    insert(grade_audit).
    values(username=user_name, date=text('getdate()'), grade_scale_id = int(grade_scale_id))
    )
        with engine.connect() as conn:
            conn.execute(insert_query)
            conn.commit()
            logging.info("Created new record")

    return_string = f'\n{input_points} points out of {max_possible_points} possible is a {letter_grade} or a {gpa}.'

    return return_string

In [264]:
get_max_grade(100,500)

"You've failed :("

In [242]:
grade_percent = .87

query = text("select top 1 * from grade_scale where grade_percent <= :x")
with engine.connect() as conn:
    logging.debug("Executing grade query")
    grade_scale = conn.execute(query, {'x': grade_percent})
    grade_scale = pd.DataFrame(grade_scale)

letter_grade = grade_scale.iloc[0,1]
gpa = grade_scale.iloc[0,3]
grade_scale_id = grade_scale.iloc[0,0]

In [243]:
grade_scale_id + 1

2

In [233]:
grade_scale_id = 5



grade_audit = table("grade_audit",
                    column("username"),
                    column("date"),
                    column("grade_scale_id"))

insert_query = (
    insert(grade_audit).
    values(username=user_name, date=text('getdate()'), grade_scale_id=grade_scale_id)
)

with engine.connect() as conn:
    conn.execute(insert_query)
    conn.commit()
